In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import os
from datetime import datetime
import math
from time import perf_counter

import random

In [8]:
class CIFARModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(CIFARModel, self).__init__(**kwargs)
        # features layer
        self.features = tf.keras.Sequential(layers=[
            layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu',
                          input_shape=(32, 32,3), name="conv1", dtype=tf.float32),
            layers.MaxPooling2D(pool_size=(2,2), name="maxpool1"),
            layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name="conv2"),
            layers.MaxPooling2D(pool_size=(2,2), name="maxpool2"),
            layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', name="conv3")
        ], name="feature")
        
        # Fully connected neural network
        self.dense = tf.keras.Sequential(layers=[
            layers.Flatten(data_format="channels_last", input_shape=(4, 4, 64)),
            layers.Dense(units=64, activation='relu'),
            layers.Dense(units=10, activation='softmax', name="output")
        ], name="dense")
    
    @tf.function
    def call(self, x):
        return self.dense(self.features(x))

    def evaluate(self, data, loss_fn, acc_fn, train=False):
        logits, labels = list(), list()
        for batch, labels_ in data:
            logits.append(self.call(batch))
            labels.append(labels_)

        y_pred = tf.concat(logits, axis=0)
        y_true = tf.concat(labels, axis=0)
        loss = loss_fn(y_true=y_true, y_pred=y_pred)

        acc = acc_fn(y_true=y_true, y_pred=y_pred)
        return acc.numpy(), loss.numpy()
    
    @tf.function
    def train_step(self, batch, labels, loss_fn, optimizer, acc_fn):
        with tf.GradientTape() as tape:
            # recored all the values on a tape in forward pass
            out = self.call(batch)
            loss = loss_fn(y_true=labels, y_pred=out)
        
        # backward pass : grad of parameters w.r.t Loss
        gradients = tape.gradient(loss, self.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(gradients, self.trainable_variables))
        acc = acc_fn(y_true=labels, y_pred=out)
        # return the loss before applying the gradients
        return loss, acc

In [32]:
def train(model, optimizer, loss_fn, acc_fn, train_data, train_size, train_batch_size, validation_data, epochs=1):   
    train_params = {'loss':[], 'accuracy': [], 'epoch':[]}
    val_params = {'loss':[], 'accuracy': [], 'epoch':[]}
    loss_arr = []  # this is for every iteration

    # we will calculate them while training the epoch
    iters = math.ceil(train_size/train_batch_size)
    
    tml = tf.metrics.Mean(name="train_loss")
    tma = tf.metrics.Mean(name="train_acc")
    global_st = perf_counter()
    for epoch in range(epochs):
        local_st = perf_counter()
        # for printing training params
        template = "Epoch:{}/{}  iter:{}/{}  loss:{:0.5f}  acc:{:0.4f}  "

        tml.reset_states()
        tma.reset_states()
        st = perf_counter()
        l_, a_ = [], []
        for i, data in enumerate(train_data):
            batch, labels = data

            # calculate the train_size and val_size if they are not none
            loss, acc = model.train_step(batch, labels, loss_fn, optimizer, acc_fn)
            loss_arr.append(loss)

            # to calculate the training loss and accuracy
            l_.append(loss)
            a_.append(acc)
            if i%50==0:
#                 print(template.format(epoch+1, epochs,
#                                      i+1,iters,
#                                      loss, acc)
#                       , end='\r')
                with tf.name_scope('per_step_training'):
                    with wtrain.as_default():
                        tf.summary.scalar("loss", loss, step=epoch*iters+i)
                        tf.summary.scalar("acc", acc, step=epoch*iters+i)
        
        # calculate the validation loss
        tml.update_state(l_)
        tma.update_state(a_)
        train_params['loss'].append(tml.result().numpy())
        train_params['accuracy'].append(tma.result().numpy())
        train_params['epoch'].append(epoch+1)

        val_acc, val_loss = model.evaluate(validation_data, loss_fn, acc_fn)
        val_params['loss'].append(val_loss)
        val_params['accuracy'].append(val_acc)
        val_params['epoch'].append(epoch+1)
        template += "val_loss:{:0.5f}  val_acc:{:0.4f} - {:0.0f}s(total: {:0.0f}s)"
        print(template.format(epoch+1, epochs,
                              i+1,iters,
                              tml.result().numpy(), tma.result().numpy(), 
                              val_loss, val_acc,
                              perf_counter()-local_st, perf_counter()-global_st))

        # write them to the tensorboard
        with tf.name_scope("per_epoch_params"):
            with wtrain.as_default():
                tf.summary.scalar("loss", tml.result().numpy(), step=epoch)
                tf.summary.scalar("acc", tma.result().numpy(), step=epoch)
                wtrain.flush()
            
            with wtest.as_default():
                tf.summary.scalar("loss", val_loss, step=epoch)
                tf.summary.scalar("acc", val_acc, step=epoch)
                wtest.flush()
            
    # return the history of training
    return train_params, val_params, loss_arr

In [33]:
model = CIFARModel(name="CIFARModel")

- Prepare the data

In [34]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [35]:
batch_size = 256

trainloader = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(batch_size).shuffle(buffer_size=50000)
test_loader = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)


In [36]:
train_size = train_images.shape[0]
train_size

50000

In [37]:
scce = tf.keras.losses.SparseCategoricalCrossentropy()
sca = tf.metrics.SparseCategoricalAccuracy()
optimizer = tf.optimizers.Adam(learning_rate=1e-3)

In [38]:
wtrain = tf.summary.create_file_writer(logdir="logs/1e-3/train")

In [39]:
wtest = tf.summary.create_file_writer(logdir="logs/1e-3/test")

In [40]:
%%time
train_params, val_params, loss_arr = train(model=model, optimizer=optimizer,
                                           loss_fn=scce, acc_fn=sca,
                                           epochs=100, train_data=trainloader,
                                           validation_data=test_loader,
                                           train_size=train_size,
                                           train_batch_size=batch_size)

Epoch:1/100  iter:196/196  loss:1.75026  acc:0.2840  val_loss:1.50510  val_acc:0.3840 - 6s(total: 6s)
Epoch:2/100  iter:196/196  loss:1.38152  acc:0.4138  val_loss:1.35461  val_acc:0.4457 - 4s(total: 10s)
Epoch:3/100  iter:196/196  loss:1.24329  acc:0.4640  val_loss:1.20499  val_acc:0.4844 - 4s(total: 14s)
Epoch:4/100  iter:196/196  loss:1.13694  acc:0.4976  val_loss:1.12545  val_acc:0.5136 - 4s(total: 17s)
Epoch:5/100  iter:196/196  loss:1.05990  acc:0.5240  val_loss:1.06190  val_acc:0.5370 - 4s(total: 21s)
Epoch:6/100  iter:196/196  loss:1.00216  acc:0.5453  val_loss:1.03135  val_acc:0.5556 - 4s(total: 25s)
Epoch:7/100  iter:196/196  loss:0.95577  acc:0.5625  val_loss:0.98428  val_acc:0.5714 - 4s(total: 29s)
Epoch:8/100  iter:196/196  loss:0.91742  acc:0.5774  val_loss:0.96920  val_acc:0.5849 - 4s(total: 33s)
Epoch:9/100  iter:196/196  loss:0.87794  acc:0.5902  val_loss:0.92543  val_acc:0.5970 - 4s(total: 36s)
Epoch:10/100  iter:196/196  loss:0.84421  acc:0.6019  val_loss:0.89974  va